# BioDendro Quick Start Example

In [1]:
#Load modules

import os
import plotly
import BioDendro

In [2]:
# Run the complete BioDendro pipeline

tree = BioDendro.pipeline("./MSMS.mgf", "./component_list.txt", clustering_method="braycurtis")

Running BioDendro v0.0.1

- input mgf file = ./MSMS.mgf
- input components file = ./component_list.txt
- neutral = False
- cutoff = 0.6
- bin_threshold = 0.0008
- clustering_method = braycurtis
- output processed file = processed.xlsx
- output results directory = results_20180605130236
- output html dendrogram = simple_dendrogram.html
- dendrogram figure width = 900
- dendrogram figure height = 400


Processing inputs
Binning and clustering
This may take some time...
Writing per-cluster summaries
Writing output html dendrogram
Finished


In [3]:
# Re-set a new cutoff for clusters

tree.cut_tree(cutoff=0.8)

In [4]:
# View the new dendrogram cutoff inline

plotly.offline.init_notebook_mode(connected=True) # for visualising plot inline
iplot = tree.iplot(width=900, height=500)
plotly.offline.iplot(iplot)

In [5]:
# Generate the out plots and tables of the new clusters.
os.makedirs("results", exist_ok=True)
tree.write_summaries(path="results")

In [6]:
# To view the number of clusters at the new cutoff

print("Cutoff:", tree.cutoff, "n clusters:", len(set(tree.clusters)))

Cutoff: 0.8 n clusters: 44


In [7]:
! BioDendro -h

usage: BioDendro [-h] [-n] [-c CUTOFF] [-b BIN_THRESHOLD]
                 [-d {jaccard,braycurtis}] [-p PROCESSED] [-o OUT_HTML]
                 [-r RESULTS_DIR] [-x WIDTH_PX] [-y HEIGHT_PX] [-q]
                 mgf components

Run the BioDendro pipeline.

positional arguments:
  mgf                   MGF input file.
  components            Listed components file.

optional arguments:
  -h, --help            show this help message and exit
  -n, --neutral         Apply neutral loss.
  -c CUTOFF, --cutoff CUTOFF
                        Distance threshold for selecting clusters from tree.
  -b BIN_THRESHOLD, --bin-threshold BIN_THRESHOLD
                        Threshold for binning m/z values prior to clustering.
  -d {jaccard,braycurtis}, --cluster-method {jaccard,braycurtis}
                        The distance metric used during tree construction.
  -p PROCESSED, --processed PROCESSED
                        Path to write preprocessed output to.
  -o OUT_HTML, --output OUT_HTML
  